In [0]:
!pip install -q tf-nightly

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

print("Tensorflow Version: {}".format(tf.__version__))
print("Eager Mode: {}".format(tf.executing_eagerly()))
print("GPU {} available".format("is" if tf.config.experimental.list_physical_devices("GPU") else "not"))

# Data Preprocessing

The `TF.data` APIs provide easier ways to load different types of files or to fetch them from the web if they can't be found on the local. 

## Load NumPy Data

In [0]:
NUMPY_DATA_URL = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz"

In [0]:
path = tf.keras.utils.get_file('mnist.npz', NUMPY_DATA_URL)
path

In [0]:
with np.load(path) as data:
  x_train = data['x_train']
  y_train = data['y_train']
  x_test = data['x_test']
  y_test = data['y_test']

In [0]:
x_train.shape, y_train.shape

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset

## Load files via Pandas

Here you are going to load a CSV file that is a small dataset for the information about heart disease. You are going to predict whether patients have heart disease or not.

In [0]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')
csv_file

In [0]:
df = pd.read_csv(csv_file)

In [0]:
df.head(5)

In [0]:
df.dtypes

In [0]:
df['thal'] = pd.Categorical(df['thal'])

Transform the categorical values. Apply the category code to the column.

In [0]:
df['thal'] = df['thal'].cat.codes

In [0]:
df.head()

## Load data via `tf.data.Dataset`

In [0]:
target = df.pop('target')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [0]:
for feat, tgt in dataset.take(5):
  print('Features: {}, Target: {}'.format(feat, tgt))

It is easy to transform a `pd.Series` object to a `np.array` or `tf.Tensor` object.

In [0]:
tf.constant(df['thal'])

You can also shuffle or batch the dataset.

In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)

In [0]:
train_dataset

# Create and Train the Model

In [0]:
def get_compiled_model():
  def build_model(inputs):
    x = tf.keras.layers.Dense(units=10, activation='elu')(inputs)
    x = tf.keras.layers.Dense(units=10, activation='elu')(x)
    y = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)
    return y

  inputs = tf.keras.Input(shape=(df.shape[1]))
  outputs = build_model(inputs)
  model = tf.keras.Model(inputs, outputs)

  model.compile(loss='binary_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()
model.summary()

In [0]:
model.fit(train_dataset, epochs=15, verbose=2)

# Create an Advanced Model

You can also select the specific columns as the inputs.

In [0]:
df.keys()

In [0]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
inputs

In [0]:
def adv_model():
  def build_model(inputs):
    x = tf.keras.layers.Dense(units=10, activation='elu')(inputs)
    x = tf.keras.layers.Dense(units=10, activation='elu')(x)
    y = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)
    return y

  inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
  x = tf.stack(list(inputs.values()), axis=-1)
  outputs = build_model(x)
  model_func = tf.keras.Model(inputs, outputs)

  model_func.compile(loss='binary_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])
  return model_func

In [0]:
model = adv_model()
model.summary()

In [0]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [0]:
df.to_dict('list').keys()

In [0]:
for dict_slice in dict_slices.take(1):
  print(dict_slice)

In [0]:
model.fit(dict_slices, epochs=15, verbose=2)